<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Table-of-Contents" data-toc-modified-id="Table-of-Contents-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Table of Contents</a></span></li></ul></div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Citation network of drafts of Chinese constitution

Hive Plot visualization

In [1]:
from pyveplot import Hiveplot, Axis, Node
import networkx as nx
import csv

G = nx.DiGraph()

with open('nodes.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')

    for i, data in enumerate(readCSV):
        if data[0] != "id":
            G.add_node(data[0], idx=i, axis=data[1], label=data[2], pos=data[3], height=data[4])

with open('edges_v2.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')

    for row in readCSV:
        if row[0] != "source":
            G.add_edge(row[0], row[1])

In [2]:
G.nodes.data()

NodeDataView({'a01': {'idx': 1, 'axis': 'Draft Articles', 'label': 'Draft 1923_1923/1', 'pos': '1', 'height': '8'}, 'a02': {'idx': 2, 'axis': 'Draft Articles', 'label': 'Draft 1923_1923/2 (1)', 'pos': '11', 'height': '8'}, 'a03': {'idx': 3, 'axis': 'Draft Articles', 'label': 'Draft 1923_1923/2 (2)', 'pos': '21', 'height': '8'}, 'a04': {'idx': 4, 'axis': 'Draft Articles', 'label': 'Draft 1923_1923/2 (3)', 'pos': '31', 'height': '8'}, 'a05': {'idx': 5, 'axis': 'Draft Articles', 'label': 'Draft 1923_1923/2 (4)', 'pos': '41', 'height': '8'}, 'a06': {'idx': 6, 'axis': 'Draft Articles', 'label': 'Draft 1923_1923/3 (1)', 'pos': '51', 'height': '8'}, 'a07': {'idx': 7, 'axis': 'Draft Articles', 'label': 'Draft 1923_1923/3 (2)', 'pos': '61', 'height': '8'}, 'a08': {'idx': 8, 'axis': 'Draft Articles', 'label': 'Draft 1923_1923/3 (3)', 'pos': '71', 'height': '8'}, 'a09': {'idx': 9, 'axis': 'Draft Articles', 'label': 'Draft 1923_1923/4', 'pos': '81', 'height': '8'}, 'a10': {'idx': 10, 'axis': 'Draf

In [3]:
colors = {}
colors[0] = ['#aae890', # bright green
             '#4daf4a', # apple green 
             '#72b077'  # greyish green
            ]
colors[1] = ['#e41a1c', # red
             '#377eb8', # cold blue
             '#984ea3', # purple
             '#ff7f00', # orange
             '#a65628', # brown
             '#f781bf'  # pink
            ]
colors[2] = ['#ffff33'] # bright yellow


# create hiveplot object
h = Hiveplot('hiveplot.svg')

# create three axes, spaced at 120 degrees from each other
h.axes = [Axis(start=20, angle=-90,
               stroke=colors[0][0], stroke_width=1),
          Axis(start=20, angle=-90 + 120,
               stroke=colors[1][0], stroke_width=1),
          Axis(start=20, angle=-90 + 120 + 120,
               stroke=colors[2][0], stroke_width=1)
          ]

# categorize them into three groups
draft_articles = [v for v in G.nodes() if G.nodes[v]['axis'] == "Draft Articles"]
foreign_leg    = [v for v in G.nodes() if G.nodes[v]['axis'] == "Foreign Legislation"]
trad_sources   = [v for v in G.nodes() if G.nodes[v]['axis'] == "Traditional Sources"]

# place these nodes onto our three axes
for axis, nodes in zip(h.axes,
                       [draft_articles, foreign_leg, trad_sources]):
    s = ""
    color_idx = -1
    circle_color = colors[2][0]
    for v in nodes:
        if axis == h.axes[0] and G.nodes[v]['label'][:10] != s :
            color_idx += 1
            circle_color = colors[0][color_idx % len(colors[0])]
            s = G.nodes[v]['label'][:10]
        if axis == h.axes[1] and G.nodes[v]['label'][:4] != s :
            color_idx +=1
            circle_color = colors[1][color_idx % len(colors[1])]
            s = G.nodes[v]['label'][:4]
        # create node object
        node = Node(radius=8,
                    label="%s" % (G.nodes[v]['label']))
        # add it to axis
        axis.add_node(v, node)
        # once it has x, y coordinates, add a circle
        node.add_circle(fill=circle_color, stroke=circle_color,
                        stroke_width=0.1, fill_opacity=0.7)
        if axis.angle < 0:
            orientation = 1
            scale = 0.6
        else:
            orientation = -1
            scale = 0.35
        # also add a label
        node.add_label(label="%s" % (G.nodes[v]['label']),
                       angle=axis.angle + 90 * orientation,
                       scale=scale)

# iterate through axes, from left to right
for n in range(-1, len(h.axes) - 1):
    # draw curves between nodes connected by edges in network
    h.connect_axes(h.axes[n],
                   h.axes[n+1],
                   G.edges,
                   stroke_width=0.3,
                   stroke='#999999')

# save output
h.save('hiveplot.svg')

In [1]:
from graph_tool.all import *

hy = Graph()
v_name = hy.new_vertex_property("string")

# The file nodes.tsv contains the hierarchy of nodes, from a single "Origin" origin, in "from"-"to" relations
with open('nodes.tsv') as csvfile: 
    readCSV = csv.reader(csvfile, delimiter='\t')

    for q, z in enumerate(readCSV):
        if len(graph_tool.util.find_vertex(hy, v_name, q)) == 0:
            s = hy.add_vertex()
            s.v_name = q
        else:
            s = graph_tool.util.find_vertex(hy, v_name, q)[0]

        if len(graph_tool.util.find_vertex(hy, v_name, z)) == 0:
            t = hy.add_vertex()
            t.v_name = z
        else:
            t = graph_tool.util.find_vertex(hy, v_name, z)[0]
        
        if not hy.edge(q, z):
            hy.add_edge(s, t)

# with open('edges_unspecific.tsv') as csvfile:
#    readCSV = csv.reader(csvfile, delimiter='\')
#
#    for row in readCSV:
#        if row[0] != "source":
#            G.add_edge(row[0], row[1])


# g.save("my_graph.xml.gz")

ModuleNotFoundError: No module named 'graph_tool'